# Выбор локации для нефтяной скважины

Необходимо провести исследоване для нефтедобывающей компании и решить, в каком из трех регионов будет выгоднее бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

**Задача**

Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap*.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

### Изучение данных

In [1]:
# Импортируем все необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
# откроем и изучим данные
data0 = pd.read_csv('geo_data_0.csv')
data1 = pd.read_csv('geo_data_1.csv')
data2 = pd.read_csv('geo_data_2.csv')

data0.info()
#data1.info()
#data2.info()

display (data0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


В трех таблицах содержатся данные геологоразведки трёх регионов. Каждый объект датасетов - характеристики скважины. По документации к данным:

* `id` - уникальный идентификатор скважины;
* `f0`, `f1`, `f2` - три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` - объём запасов в скважине (тыс. баррелей).

В данных нет пропусков и все значения численные, масштаб примерно одинаков. Столбец с уникальным идентификатором можно удалить, чтобы не мешать модели обучаться. Потом мы сможем восстановить ID по индексу объекта.


**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
* Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

In [3]:
def dropid(data):
    df = data.drop('id', axis=1)
    return df
df0 = dropid(data0)
df1 = dropid(data1)
df2 = dropid(data2)

## Обучение и проверка модели

### Разделение на выборки

Целевой признак - объем запасов в скважинах.

In [4]:
# разделим три выборки на валидационную и тренировочкую 25:75
def split(data):
    features = data.drop('product', axis=1)
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

features_train_0, features_valid_0, target_train_0, target_valid_0 = split(df0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = split(df1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = split(df2)

features_train_2.head()

,f0,f1,f2
27212,-0.906828,1.339033,1.106789
7866,-1.536353,-0.704673,-1.747632
62041,-1.959584,0.356044,3.528659
70185,2.129734,2.713459,-0.156172
82230,-0.332026,1.515604,5.417583


### Обучение модели

По условиям задачи для обучения модели подходит только линейная регрессия, так как остальные — недостаточно предсказуемые.

<b>Обучим модель на регионах.</b>

In [5]:
# обучим модель и сделаем предсказания на валидационной выборке
data = [df0, df1, df2]
features_train = [features_train_0, features_train_1, features_train_2]
features_valid = [features_valid_0, features_valid_1, features_valid_2]
target_train = [target_train_0, target_train_1, target_train_2]
target_valid = [target_valid_0, target_valid_1, target_valid_2]
regions = ['Первый регион', 'Второй регион', 'Третий регион']

predictions = []
targets = []
for i in range(len(data)):
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])
    
    predictions.append(model.predict(features_valid[i]))
    targets.append(target_valid[i])
    
    print (regions[i])
    print ('Средний запас предсказанного сырья: ', round(predictions[i].mean(), 2), 'тыс. баррелей')
    print ('RMSE модели: ', round((mean_squared_error(targets[i], predictions[i]) ** 0.5), 2), 'тыс. баррелей')
    print (' ')

Первый регион
Средний запас предсказанного сырья:  92.59 тыс. баррелей
RMSE модели:  37.58 тыс. баррелей
 
Второй регион
Средний запас предсказанного сырья:  68.73 тыс. баррелей
RMSE модели:  0.89 тыс. баррелей
 
Третий регион
Средний запас предсказанного сырья:  94.97 тыс. баррелей
RMSE модели:  40.03 тыс. баррелей
 


<b>Вывод</b>

Мы получили предсказания запасов на трех регионах, средняя ошибка по трем регионам ~55 тыс баррелей. Наиболее прибыльный - третий регион - средний запас ~ 95 тыс баррелей с одной скважины. 

## Подготовка к расчёту прибыли

In [6]:
# точки, которые исследуют при разведке региона
explore = 500

# лучшие точки для разработки по результатам МО
best_explore = 200

# бюджет на разработку 200 скважин в регионе
budget_for_wells = 10000000000

# доход с каждой единицы продукта
revenue = 450000

min_product = budget_for_wells / best_explore / revenue

print ('Минимальный объем сырья для безубыточной разработки скважины: ', round(min_product), 'тыс. баррелей')

Минимальный объем сырья для безубыточной разработки скважины:  111 тыс. баррелей


<b> Вывод </b>

Для того чтобы регион был безубыточный, необходимо, чтобы в регионе запасы потенциального сырья для выкачки были не менее 111 тыс. баррелей. Что на ~16 единиц превышает средний запас региона с самым большим количеством запасов.

In [7]:
# преобразуем предсказания моделей и целевые признаки 
target_0 = pd.Series(targets[0]).reset_index(drop=True)
prediction_0 = pd.Series(predictions[0])

target_1 = pd.Series(targets[1]).reset_index(drop=True)
prediction_1 = pd.Series(predictions[1])

target_2 = pd.Series(targets[2]).reset_index(drop=True)
prediction_2 = pd.Series(predictions[2])

Составим функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [8]:
def count_revenue(target, prediction, count):

    # отсортируем предсказания и выделим скважины с максимальными значениями предсказаний
    pred_sorted = prediction.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    
    # функция возвращает суммарную прибыль с отобранных скважин
    return revenue * selected.sum()

#протестируем работу функции
print ('Суммарная прибыль с 200 лучших скважин в первом регионе: ', round(
    count_revenue(target_0, prediction_0, best_explore)), 'тыс. баррелей')

Суммарная прибыль с 200 лучших скважин в первом регионе:  13320826043 тыс. баррелей


## Расчёт прибыли и рисков 

<b>Посчитаем риски и прибыль с каждого региона</b>


In [9]:
# создадим списки для трех регионов
target = [target_0, target_1, target_2]
predictions = [prediction_0, prediction_1, prediction_2]
regions = ['Первый регион', 'Второй регион', 'Третий регион']

state = np.random.RandomState(12345)

for j in range(len(target)):
    values = []
    for i in range (1000):   
        
        # выделим 1000 разных выборок по 500 скважин
        target_subsample = target[j].sample(n=explore, replace=True, random_state=state)
        pred_subsample = predictions[j].loc[target_subsample.index]
        
        # посчитаем доход с 200 самых прибыльных
        values.append(count_revenue(target_subsample, pred_subsample, best_explore))
        
    values = pd.Series(values)
    
    # посчитаем среднюю прибыль и доверительный интервал
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    # посчитаем риск убытков
    risk = (values < budget_for_wells).mean()
       
    
    print (regions[j])
    print ('Средняя прибыль: ', round((mean / 1000000000), 3), 'млрд. баррелей')
    print ('95% доверительный интервал: от ', round((lower / 1000000000), 3), 'млрд. баррелей до ',
           round((upper / 1000000000), 3), 'млрд. баррелей')  
    print ('Риск убытков: {:.1%}'.format (risk))
    print (' ')


Первый регион
Средняя прибыль:  10.426 млрд. баррелей
95% доверительный интервал: от  9.898 млрд. баррелей до  10.948 млрд. баррелей
Риск убытков: 6.0%
 
Второй регион
Средняя прибыль:  10.518 млрд. баррелей
95% доверительный интервал: от  10.128 млрд. баррелей до  10.954 млрд. баррелей
Риск убытков: 0.3%
 
Третий регион
Средняя прибыль:  10.42 млрд. баррелей
95% доверительный интервал: от  9.884 млрд. баррелей до  10.99 млрд. баррелей
Риск убытков: 6.2%
 


<b>Вывод</b>

Несмотря на то, что наибольший средний запас предсказанного сырья находится в третьем регионе, а наименьшую среднюю ошибку показала модель первого региона, самым выгодным регионом для разработки скважин является <b>второй регион</b>. В данном регионе при разведке 500 точек и разрабоке 200 лучших из них с вероятностью 95% средняя прибыль будет самая высокая - <b>10 млрд 510 млн рублей</b>. Риск убытков в данном регионе - <b>0.3%</b>.

В исследовании выбора локации для скважин для поставленной задачи: определить регион, где добыча принесёт наибольшую прибыль, было выполнено следующее:

1. Подготовленны данные:
* Изучены csv таблицы
* Сделаны выводы
2. Модели обучены
* Разделены на выборки: тренировочную и валидационную
* Модели обучены на линейной регрессии
* Выявлены средние запасы предсказанных объемов нефти и RMSE моделей
3. Подготовленны данные к расчету прибыли
* Минимальный объем сырья для безубыточной разработки скважины:  111 тыс. баррелей
* Предсказания и целевой признак преобразованы
* Составлена функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
4. Расчитана средняя прибыль, доверительный интервал и риски
* Посчитана средняя прибыль, доверительный интервал, риски для каждого региона
* Выявлен оптимальный регион для добычи нефти - Второй регион